In [ ]:
import torch
import numpy as np
import evaluate
from datasets import load_dataset
from transformers import (
    AutoImageProcessor, DefaultDataCollator,
    AutoModelForImageClassification, TrainingArguments, Trainer, pipeline
)
from torchvision.transforms import RandomResizedCrop, Compose, Normalize, ToTensor

## Load Food-101 dataset

In [ ]:
food = load_dataset("food101", split="train[:5000]")

In [ ]:
food = food.train_test_split(test_size=0.2)

In [ ]:
example = food["train"][0]

In [ ]:
example['image']

In [ ]:
labels = food["train"].features["label"].names
label2id, id2label = dict(), dict()
for i, label in enumerate(labels):
    label2id[label] = str(i)
    id2label[str(i)] = label

In [ ]:
id2label[str(79)]

## Preprocess

In [ ]:
checkpoint = "google/vit-base-patch16-224-in21k"
image_processor = AutoImageProcessor.from_pretrained(checkpoint)

In [ ]:
normalize = Normalize(mean=image_processor.image_mean, std=image_processor.image_std)
size = (
    image_processor.size["shortest_edge"]
    if "shortest_edge" in image_processor.size
    else (image_processor.size["height"], image_processor.size["width"])
)
_transforms = Compose([RandomResizedCrop(size), ToTensor(), normalize])

In [ ]:
def transforms(examples):
    examples["pixel_values"] = [_transforms(img.convert("RGB")) for img in examples["image"]]
    del examples["image"]
    return examples

In [ ]:
food = food.with_transform(transforms)

In [ ]:
data_collator = DefaultDataCollator()

## Model

In [ ]:
model = AutoModelForImageClassification.from_pretrained(
    "resnet_model", trust_remote_code=True
)

## Evaluate

In [ ]:
accuracy = evaluate.load("accuracy")

In [ ]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

## Train

In [ ]:
model = AutoModelForImageClassification.from_pretrained(
    checkpoint,
    num_labels=len(labels),
    id2label=id2label,
    label2id=label2id,
)

In [ ]:
training_args = TrainingArguments(
    output_dir="my_awesome_food_model",
    remove_unused_columns=False,
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=16,
    gradient_accumulation_steps=4,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    warmup_ratio=0.1,
    logging_steps=10,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    push_to_hub=False,
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=food["train"],
    eval_dataset=food["test"],
    tokenizer=image_processor,
    compute_metrics=compute_metrics,
)

trainer.train()

## inference

In [ ]:
ds = load_dataset("food101", split="validation[:10]")
image = ds["image"][0]

In [ ]:
image

In [ ]:
classifier = pipeline("image-classification", model="my_awesome_food_model/checkpoint-186")
classifier(image)

In [ ]:
image_processor = AutoImageProcessor.from_pretrained("my_awesome_food_model/checkpoint-186")
inputs = image_processor(image, return_tensors="pt")

In [ ]:
model = AutoModelForImageClassification.from_pretrained("my_awesome_food_model/checkpoint-186")
with torch.no_grad():
    logits = model(**inputs).logits

In [ ]:
logits

In [ ]:
predicted_label = logits.argmax(-1).item()
model.config.id2label[predicted_label]